# twStock notebook

------


## 目的
1. 擷取台股即時資料分析
2. 分析個股定期不定額
3. **注意台股TWSE 有request limit** _(每5秒鐘3個request,過的話會被封鎖ip至少20分鐘以上)_
4. https://www.twse.com.tw/zh/
------

# 第一金
* 畫出十年 股票價 及 成交量

In [2]:
import twstock
from twstock import Stock
from twstock import BestFourPoint
import pandas as pd
import numpy as np
import matplotlib as plt
import time
import datetime

start = time.time()

# time.sleep(5)                  # request time limit
stock = Stock('2892')          # 擷取第一金股價
stock.fetch_from(2015, 1)

done = time.time()
elapsed = int(done - start)
print('Time used: '+ str(elapsed) + 'sec for request time limit')

ConnectionError: HTTPConnectionPool(host='www.twse.com.tw', port=80): Max retries exceeded with url: /exchangeReport/STOCK_DAY?date=20181001&stockNo=2892 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EC0FE60F28>: Failed to establish a new connection: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。',))

#  瑞耘
* 四大買賣點分析

In [ ]:
import twstock
from twstock import Stock
from twstock import BestFourPoint
import pandas as pd
import numpy as np

print(twstock.codes['6532'].name)
print(twstock.codes['6532'].start)
print(twstock.codes['6532'])   

stock = Stock('6532')               # 擷取股價
bfp = BestFourPoint(stock)
print('\n判斷是否為四大 買點')
print(bfp.best_four_point_to_buy())    # 判斷是否為四大買點
print('\n判斷是否為四大 賣點')
print(bfp.best_four_point_to_sell())   # 判斷是否為四大賣點
print('\n綜合判斷')
print(bfp.best_four_point())           # 綜合判斷

ma_p = stock.moving_average(stock.price, 5)       # 計算五日均價
ma_c = stock.moving_average(stock.capacity, 5)    # 計算五日均量
ma_p_cont = stock.continuous(ma_p)                # 計算五日均價持續天數
ma_br = stock.ma_bias_ratio(5, 10)                # 計算五日、十日乖離值
d = {'ma_p': ma_p, 'ma_c': ma_c}
df = pd.DataFrame(data=d)
print('\n計算五日均價持續天數:')
print(ma_p_cont)
print('\n計算 ma_p(五日均價) ma_c(五日均量):')
print(df)

twstock.realtime.get('6532')    # 擷取當前股票資訊
twstock.realtime.get(['6532'])  # 擷取當前三檔資訊

## 基本操作
* 匯入twstock library:

In [ ]:
from twstock import Stock

stock = Stock('2892')                             # 擷取第一金股價
ma_p = stock.moving_average(stock.price, 5)       # 計算五日均價
ma_c = stock.moving_average(stock.capacity, 5)    # 計算五日均量
ma_p_cont = stock.continuous(ma_p)                # 計算五日均價持續天數
ma_br = stock.ma_bias_ratio(5, 10)                # 計算五日、十日乖離值

In [ ]:
ma_p_cont

In [ ]:
import pandas as pd
import numpy as np
d = {'ma_p': ma_p, 'ma_c': ma_c}
df = pd.DataFrame(data=d)
df

In [ ]:
import pandas as pd
import numpy as np

d = {'ma_br': ma_br}
df = pd.DataFrame(data=d)
df

* 擷取自 2015 年 1 月至今之資料

In [ ]:
stock.fetch_from(2015, 1)

* 基本資料之使用:

In [ ]:
stock.price

In [ ]:
stock.capacity

In [ ]:
stock.data[0]

------

## 附件：Juypter Notebook 基本操作 

### Jupyter Notebook 基本操作介紹影片:

In [ ]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/HW29067qVWk" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

------
### Juypter % 及 %% 外掛程式運用:

In [ ]:
%lsmagic

### 圖表基本操作 (matplot library):

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

N = 50
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
area = np.pi * (15 * np.random.rand(N))**2

plt.scatter(x, y, s=area, c=colors, alpha=0.5)
plt.show()

### 計算時間:

In [ ]:
%%timeit
square_evens = [n*n for n in range(1000)]

### 資料呈現(panda):

* 解決ImportError: cannot import name 'nosetester'問題:
  * numpy 1.11.1 version
    * pip3 uninstall numpy
    * pip3 install numpy==1.11.1
  * pandas 0.19.2 version 
    * pip3 uninstall pandas
    * pip3 install pandas==0.19.2

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(np.random.randn(10,5))
df.head()
